In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds

In [2]:
import urllib3
urllib3.disable_warnings()

#tfds.disable_progress_bar()   # 이 주석을 풀면 데이터셋 다운로드과정의 프로그레스바가 나타나지 않습니다.

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
)

In [3]:
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    # image = tf.image.resize(image, [32, 32])
    return tf.cast(image, tf.float32) / 255., label

In [4]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [5]:
class ConvBlock(tf.keras.Model):
    def __init__(self, channel, kernel_size, strides = 1):
        super(ConvBlock,self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(channel, kernel_size=3, strides=strides, padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()
        
    def call(self, input_tensor, training = False):
        x = self.conv1(input_tensor)
        x = self.bn1(x, training = training)
        return x

In [6]:
class SELayer(tf.keras.layers.Layer):
    def __init__(self,out_channel =64, r_ratio=4):
        super(SELayer,self).__init__()
        self.gap = tf.keras.layers.GlobalAveragePooling2D()
        self.fc1 = tf.keras.layers.Dense(out_channel//r_ratio)
        self.fc2 = tf.keras.layers.Dense(out_channel)
        self.output_channel = out_channel
        self.reshape = tf.keras.layers.Reshape((1,1,out_channel))
    
    def call(self,input_tensor,training = False):
        x = self.gap(input_tensor)
        x = self.fc1(x)
        x = tf.nn.relu(x)
        x = self.fc2(x)
        x = tf.nn.sigmoid(x)
        x = self.reshape(x)
        return x*input_tensor

In [7]:
class SEBlock(tf.keras.layers.Layer):
    def __init__(self,channel,down_sample = False, r_ratio=4):
        super(SEBlock,self).__init__()
        self.down_sample = down_sample
        
        if self.down_sample:
            self.conv_block1 = ConvBlock(channel,kernel_size=1, strides=2)
        else:
            self.conv_block1 = ConvBlock(channel,kernel_size=1, strides=1)
        
        self.conv_block2 = ConvBlock(channel,kernel_size=3, strides=1)     
        self.conv_block3 = ConvBlock(channel*4, kernel_size=1, strides=1)
        
        self.se1 = SELayer(channel*4,r_ratio)
                
        if self.down_sample:
            self.downsample_conv = tf.keras.layers.Conv2D(channel*4, kernel_size=1, strides=2, padding='same')
            self.downsample_bn = tf.keras.layers.BatchNormalization()
                
    def call(self,input_tensor,training = False):
        x = self.conv_block1(input_tensor)
        x = tf.nn.relu(x)
        x = self.conv_block2(x)
        x = tf.nn.relu(x)
        x = self.conv_block3(x)
        x = self.se1(x)
        
        if self.down_sample:
            input_tensor = self.downsample_conv(input_tensor)
            input_tensor = self.downsample_bn(input_tensor, training= training)
        
        x = x + input_tensor
        x = tf.nn.relu(x)
        return x

In [8]:
class SE_ResNet50(tf.keras.Model):
    def __init__(self):
        super(SE_ResNet50,self).__init__()
        
        self.conv = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')
        self.bn = tf.keras.layers.BatchNormalization()     
        self.maxpool = tf.keras.layers.MaxPool2D(pool_size=2)
               
        self.conv1  = tf.keras.Sequential()
        self.conv2  = tf.keras.Sequential()
        self.conv3  = tf.keras.Sequential()
        self.conv4  = tf.keras.Sequential()
        
        for i in range(3):
            if i==0:
                self.conv1.add(SEBlock(64, down_sample = True))
            else:
                self.conv1.add(SEBlock(64))

        for i in range(4):
            if i==0:
                self.conv2.add(SEBlock(128, down_sample = True))
            else:
                self.conv2.add(SEBlock(128))

        for i in range(6):
            if i==0:
                self.conv3.add(SEBlock(256, down_sample = True))
            else:
                self.conv3.add(SEBlock(256))

        for i in range(3):
            if i==0:
                self.conv4.add(SEBlock(512, down_sample = True))
            else:
                self.conv4.add(SEBlock(512))

        self.gap = tf.keras.layers.GlobalAveragePooling2D()
        self.fc1 = tf.keras.layers.Dense(4096, activation='relu')
        self.fc2 = tf.keras.layers.Dense(4096, activation='relu')
        self.fc3 = tf.keras.layers.Dense(10, activation='softmax')        
        
    def call(self, input_tensor, training = False):
        x = self.conv(input_tensor)
        x = self.bn(x)
        x = tf.nn.relu(x)
        x = self.maxpool(x)

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        
        x = self.gap(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [9]:
BATCH_SIZE = 256
EPOCH = 20

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)
ds_train = apply_normalize_on_dataset(ds_train, batch_size=BATCH_SIZE)
ds_test = apply_normalize_on_dataset(ds_test, batch_size=BATCH_SIZE)

In [10]:
se_resnet50 = SE_ResNet50()

se_resnet50.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_se_resnet50 = se_resnet50.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1
)

Epoch 1/20
195/195 [==============================] - 28s 144ms/step - loss: 2.0929 - accuracy: 0.2293 - val_loss: 2.3911 - val_accuracy: 0.1048
Epoch 2/20
195/195 [==============================] - 26s 132ms/step - loss: 1.7855 - accuracy: 0.3517 - val_loss: 2.5329 - val_accuracy: 0.1382
Epoch 3/20
195/195 [==============================] - 26s 132ms/step - loss: 1.6176 - accuracy: 0.4146 - val_loss: 1.7372 - val_accuracy: 0.3636
Epoch 4/20
195/195 [==============================] - 26s 132ms/step - loss: 1.4582 - accuracy: 0.4766 - val_loss: 1.6015 - val_accuracy: 0.4218
Epoch 5/20
195/195 [==============================] - 26s 133ms/step - loss: 1.3083 - accuracy: 0.5327 - val_loss: 1.6081 - val_accuracy: 0.4301
Epoch 6/20
195/195 [==============================] - 26s 133ms/step - loss: 1.1670 - accuracy: 0.5856 - val_loss: 1.6211 - val_accuracy: 0.4371
Epoch 7/20
195/195 [==============================] - 26s 131ms/step - loss: 0.9600 - accuracy: 0.6670 - val_loss: 1.7055 - val_ac